In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *

In [2]:
PATH="/home/kirana/Documents/phd"
DATAPATH="/home/kirana/Documents/phd/data/experiment/SST_2"

## Dataset

In [3]:
[df_train,df_valid,itos, train_tokens, valid_tokens, trn_lm, val_lm]=pickle.load(open(f'{DATAPATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [4]:
type(df_train)

pandas.core.frame.DataFrame

In [5]:
df_train.head()

,text,label,dstype,words,tokens
55236,exactly wrong,1,train,"[ \n , xxbos, xxfld, 1, exactly, wrong]","[3, 4, 5, 2, 628, 459]"
100531,the brain,0,train,"[ \n , xxbos, xxfld, 1, the, brain]","[3, 4, 5, 2, 7, 808]"
90848,routine action and jokes like this,1,train,"[ \n , xxbos, xxfld, 1, routine, action, and, ...","[3, 4, 5, 2, 1036, 117, 10, 389, 37, 27]"
17512,"If you like quirky , odd movies and\/or the ir...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, if, you, like, ...","[3, 4, 5, 2, 6, 65, 29, 37, 382, 8, 1037, 111,..."
107105,to accomplish what few sequels can,0,train,"[ \n , xxbos, xxfld, 1, to, accomplish, what, ...","[3, 4, 5, 2, 13, 7436, 64, 186, 1456, 72]"


In [6]:
df=pd.concat([df_train,df_valid],axis=0)

In [7]:
df['dstype'].value_counts()

train    117220
test       1821
valid       872
Name: dstype, dtype: int64

In [8]:
df_train=df.loc[df['dstype']!='test']
df_test=df.loc[df['dstype']=='test']

In [9]:
df_train.shape, df_test.shape

((118092, 5), (1821, 5))

In [10]:
df_train.head()

,text,label,dstype,words,tokens
55236,exactly wrong,1,train,"[ \n , xxbos, xxfld, 1, exactly, wrong]","[3, 4, 5, 2, 628, 459]"
100531,the brain,0,train,"[ \n , xxbos, xxfld, 1, the, brain]","[3, 4, 5, 2, 7, 808]"
90848,routine action and jokes like this,1,train,"[ \n , xxbos, xxfld, 1, routine, action, and, ...","[3, 4, 5, 2, 1036, 117, 10, 389, 37, 27]"
17512,"If you like quirky , odd movies and\/or the ir...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, if, you, like, ...","[3, 4, 5, 2, 6, 65, 29, 37, 382, 8, 1037, 111,..."
107105,to accomplish what few sequels can,0,train,"[ \n , xxbos, xxfld, 1, to, accomplish, what, ...","[3, 4, 5, 2, 13, 7436, 64, 186, 1456, 72]"


In [11]:
type(df_train['tokens'])

pandas.core.series.Series

In [12]:
bs=52 # 52 - Jeremey, 20 - default

In [13]:
bptt= 70 #70 - Jeremey, 35 - default

In [14]:
df_train.head()

,text,label,dstype,words,tokens
55236,exactly wrong,1,train,"[ \n , xxbos, xxfld, 1, exactly, wrong]","[3, 4, 5, 2, 628, 459]"
100531,the brain,0,train,"[ \n , xxbos, xxfld, 1, the, brain]","[3, 4, 5, 2, 7, 808]"
90848,routine action and jokes like this,1,train,"[ \n , xxbos, xxfld, 1, routine, action, and, ...","[3, 4, 5, 2, 1036, 117, 10, 389, 37, 27]"
17512,"If you like quirky , odd movies and\/or the ir...",0,train,"[ \n , xxbos, xxfld, 1, xxmaj, if, you, like, ...","[3, 4, 5, 2, 6, 65, 29, 37, 382, 8, 1037, 111,..."
107105,to accomplish what few sequels can,0,train,"[ \n , xxbos, xxfld, 1, to, accomplish, what, ...","[3, 4, 5, 2, 13, 7436, 64, 186, 1456, 72]"


In [15]:
df_train['label'].value_counts()

0    64559
1    53533
Name: label, dtype: int64

In [16]:
df_valid['label'].value_counts()

0    6614
1    5378
Name: label, dtype: int64

In [17]:
itos[:5]

['_unk_', '_pad_', '1', ' \n ', 'xxbos']

In [18]:
padlen=1400
padding_idx=1

In [19]:
df_train['n_tok']=df_train['tokens'].apply(len)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_train['n_tok'].describe()

count    118092.000000
mean         14.795202
std           9.114027
min           5.000000
25%           8.000000
50%          12.000000
75%          19.000000
max          66.000000
Name: n_tok, dtype: float64

In [21]:
df_train.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [23]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
0
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
0
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
1
1400
[3. 4. 5. 2. ... 1. 1. 1. 1.]
0
1400


In [25]:
df_train.shape

(118092, 6)

In [26]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(0    64559
 1    53533
 Name: label, dtype: int64, 0    6614
 1    5378
 Name: label, dtype: int64, 1    912
 0    909
 Name: label, dtype: int64)

In [27]:
df_train.shape, df_valid.shape

((118092, 6), (11992, 6))

In [28]:
padlen

1400

In [29]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [30]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [31]:
padlen

1400

In [32]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [33]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [34]:
for xb,yb,xlen in dltrain:
    break

In [35]:
xb, yb, xlen

(tensor([[3, 4, 5,  ..., 1, 1, 1],
         [3, 4, 5,  ..., 1, 1, 1],
         [3, 4, 5,  ..., 1, 1, 1],
         ...,
         [3, 4, 5,  ..., 1, 1, 1],
         [3, 4, 5,  ..., 1, 1, 1],
         [3, 4, 5,  ..., 1, 1, 1]]),
 tensor([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
         1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
         0, 0, 0, 0]),
 tensor([12, 12, 15,  6,  5,  7, 12,  7, 19, 23, 51, 12,  9,  8, 40, 11, 12, 13,
         18,  8,  6,  8, 11, 17, 29,  6,  6, 11, 32, 11,  7, 30,  6,  8,  8, 10,
         17, 11, 22, 13,  6,  5, 10,  8, 31, 10, 13, 13,  8, 11, 34, 19]))

## Model Architecture

In [36]:
from torch import nn
import torch
from torch.autograd import Variable

In [37]:
bs,bptt

(52, 70)

In [38]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5 - changing to 0.4, 0.3 or any dropout value did not make much difference
dropout_o=0.5 #0.5
n_out=1

In [39]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [40]:
df_train['label'].value_counts()

0    64559
1    53533
Name: label, dtype: int64

In [41]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [42]:
itos[:5]

['_unk_', '_pad_', '1', ' \n ', 'xxbos']

In [43]:
import torch

In [44]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [45]:
m(input).size()

torch.Size([1, 64, 5])

In [46]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_out)
        

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc(big_out)

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [47]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [48]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [49]:
device="cpu"

## Load pretrained model weights

In [50]:
pretrained_lm_weights=pickle.load(open(f'{DATAPATH}/inter/varybpttpretrained_lm_weights','rb'))

### Check if model forward works

In [51]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(16206, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [52]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (16206, 400)


In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 13,377,401 trainable parameters


In [54]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(16206, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=1900, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [55]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.0913,  0.2028,  0.1955,  0.0666,  0.0973,  0.1095,  0.1677,  0.1011,
          0.2282,  0.0662,  0.1162,  0.1889,  0.3111,  0.3090,  0.3414,  0.1239,
          0.2064,  0.1587,  0.3076,  0.2383,  0.1050,  0.2128,  0.1956,  0.2649,
          0.0613,  0.0504, -0.0365,  0.0047,  0.2606,  0.0542,  0.1052,  0.1514,
          0.0614,  0.1064,  0.1071,  0.1445,  0.1342,  0.2717,  0.1774,  0.2164,
          0.1520,  0.1030, -0.0641,  0.2737,  0.4493,  0.0790,  0.2542,  0.1322,
          0.1005,  0.2995,  0.0244,  0.2588], grad_fn=<ViewBackward>),
 tensor(0.7113, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [56]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [57]:
preds.view(-1)

tensor([ 0.2081,  0.2063,  0.2703,  0.1055,  0.1278,  0.1643,  0.1241,  0.1315,
         0.1483,  0.1771,  0.2232,  0.1870,  0.1492,  0.0223,  0.2462,  0.1787,
         0.0300,  0.1415,  0.3247,  0.1125,  0.1255,  0.2631,  0.2229,  0.2371,
         0.0840, -0.0616,  0.1579,  0.0863,  0.2747,  0.2379,  0.2112,  0.1756,
         0.0942,  0.0804,  0.0555,  0.2014,  0.3159,  0.2950,  0.2507,  0.1369,
         0.0615,  0.0718,  0.2729,  0.1516, -0.0664,  0.1887,  0.1910,  0.2037,
         0.1685,  0.3118,  0.3335,  0.1391], grad_fn=<ViewBackward>)

In [58]:
yb

tensor([0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0])

In [59]:
preds.size()

torch.Size([52])

In [60]:
yb.size()

torch.Size([52])

In [61]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.4423)

In [62]:
from sklearn.metrics import roc_auc_score

In [63]:
roc_auc_score(yb,preds)

0.4761904761904762

In [64]:
pretrained_lm_weights

array([[-0.01301 , -0.105954, -0.285109, -0.044226, ..., -0.053858, -0.01929 ,  0.043896,  0.068098],
       [ 0.00827 , -0.032289,  0.043977, -0.048435, ..., -0.179876,  0.008289, -0.005907, -0.028031],
       [-0.23123 ,  0.223312, -0.131881,  0.020966, ..., -0.048621,  0.039325,  0.008791,  0.165966],
       [ 0.094656,  0.297782,  0.098514,  0.054791, ..., -0.028909, -0.073314, -0.230205,  0.217315],
       ...,
       [ 0.300145, -0.141749,  0.076261,  0.071102, ..., -0.198638,  0.188576,  0.521759,  0.338996],
       [-0.29253 , -0.366779, -0.062945, -0.49639 , ..., -0.151594, -0.152283,  0.324542, -0.003129],
       [-0.12251 , -0.28435 , -0.002887,  0.009289, ...,  0.321684,  0.055438, -0.03748 , -0.035654],
       [ 0.398688, -0.061674, -0.069627, -0.229945, ..., -0.649922, -0.712466, -0.19228 , -0.392854]], dtype=float32)

## Learner

In [65]:
# Weight Decay Schedule
tempstart=1e-7
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*6

0 1e-07
1 6e-07
2 3.6e-06
3 2.16e-05
4 0.0001296
5 0.0007776


In [66]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*(0.7**i)

0 0.005
1 0.005
2 0.0034999999999999996
3 0.0017149999999999995
4 0.0005882449999999997
5 0.0001412376244999999


In [67]:
torch.randint(1,5,(1,))

tensor([3])

In [68]:
lrs=[1e-2,5e-3,1e-4,5e-4]

In [69]:
torch.randint(0,4,(1,))

tensor([1])

In [70]:
lrs[torch.randint(0,4,(1,))]

0.0001

In [71]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.7,wd_mult=6):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        self.lrs=[1e-2,5e-3,1e-4,5e-4]
        self.preds,self.preds_valid,self.trainY,self.actual=[],[],[],[]
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            
            
            if mode_train:
                self.trainY.append(Yb.view(-1))
                self.preds.append(preds.data)
            else:
                self.actual.append(Yb.view(-1))
                self.preds_valid.append(preds.data)

            
            del preds
        
        if mode_train:
            if 1==0:
                lr =self.lrs[torch.randint(0,4,(1,))]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=self.n_epoch+reset_cycle
                else:
                    self.lr*=(self.lr_decay**self.n_epoch)  
                    if self.n_epoch>1:
                        self.wd*=self.wd_mult
            self.n_epoch+=1
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                #param_group['weight_decay']=self.wd

In [72]:
device="cuda:0"

In [73]:
n_layers

2

In [74]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (16206, 400)


In [75]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
#optimizer=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)
metric_fn=accuracy_binomial

In [76]:
len(dltrain),len(dlvalid)

(2271, 231)

In [77]:
df_valid['label'].value_counts()

0    6614
1    5378
Name: label, dtype: int64

In [78]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,100,0.25,cycle_mult=6)

In [79]:
model_sentiment.freeze_embedding()

In [80]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [81]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [82]:
learner.run_epochs(dltrain,dlvalid,5)

Batch:100 0.5909485232830047  0.6980769488215447
Batch:200 0.5485953556001186  0.7303846408426762
Batch:300 0.525516549795866  0.7436538725097974
Batch:400 0.49789963245391844  0.7605769506841898
Batch:500 0.4850561005473137  0.768423104584217
Batch:600 0.4723142488549153  0.7769872075815996
Batch:700 0.4634835795845304  0.7825275009444782
Batch:800 0.45274866512045264  0.7884375285729766
Batch:900 0.4439710036913554  0.79331199483739
Batch:1000 0.4381228656321764  0.7971538756787777
Batch:1100 0.4309715035557747  0.801206323531541
Batch:1200 0.42554636576523386  0.804775671089689
Batch:1300 0.42124695611687807  0.8076035802868696
Batch:1400 0.4151538156185831  0.8103434365774904
Batch:1500 0.41074812969565394  0.8128974660038948
Batch:1600 0.408701460454613  0.8141707031987607
Batch:1700 0.40425703863010687  0.8165271797425606
Batch:1800 0.40019029789500765  0.8185470390154256
Batch:1900 0.3971103906317761  0.8204453747366605
Batch:2000 0.3940261724367738  0.822076953843236
Batch:2100

In [83]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_ytsm')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo9_0.ytsm')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo9_ytsm')
torch.save (optimizer,f'{DATAPATH}/inter/combo9_ytsm')
torch.save (learner,f'{DATAPATH}/inter/combo9_ytsm')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [84]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:100 0.1423038242198527  0.9440384972095489
Batch:200 0.14198769419454038  0.9450961890816688
Batch:300 0.13988345813627046  0.9464743942022323
Batch:400 0.14080730407964437  0.9462019582092762
Batch:500 0.1387798697091639  0.946884650349617
Batch:600 0.13860795518072944  0.9465705478191375
Batch:700 0.1391347512842289  0.9460989363704408
Batch:800 0.13816526560811326  0.9467548429965973
Batch:900 0.13883872881738676  0.9464743942684597
Batch:1000 0.13895554983057082  0.9462884968519211
Batch:1100 0.13851418417963116  0.946293741681359
Batch:1200 0.13755214336172988  0.946570548315843
Batch:1300 0.13789078454701947  0.9463609821979816
Batch:1400 0.137262621938384  0.9467720135194915
Batch:1500 0.13671974578251442  0.9469615738391877
Batch:1600 0.1361846348445397  0.9470072471350431
Batch:1700 0.13698934907422347  0.9465498093647116
Batch:1800 0.1366460019350052  0.9468162747886446
Batch:1900 0.1369487076233092  0.9466599545980755
Batch:2000 0.13681243092752993  0.946740420371294
B

In [85]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/model_sentiment_state_dict')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/optimizer_state_dict')
torch.save (model_sentiment,f'{DATAPATH}/inter/model_sentiment')
torch.save (optimizer,f'{DATAPATH}/inter/optimizer')
torch.save (learner,f'{DATAPATH}/inter/learner')

In [94]:
model_sentiment=torch.load (f'{DATAPATH}/inter/model_sentiment_state_dict')
optimizer=torch.load (f'{DATAPATH}/inter/optimizer_state_dict')
learner=torch.load (f'{DATAPATH}/inter/model_sentiment')

In [96]:
model_sentiment.wd

0.0001296

In [97]:
from itertools import chain
preds=list(chain.from_iterable(learner.preds))[-df_train.shape[0]:]
preds_valid=list(chain.from_iterable(learner.preds_valid))[-df_valid.shape[0]:]
trainY=list(chain.from_iterable(learner.trainY))[-df_train.shape[0]:]
actual=list(chain.from_iterable(learner.actual))[-df_valid.shape[0]:]
preds=[x.item() for x in preds]
preds_valid=[x.item() for x in preds_valid]
trainY=[x.item() for x in trainY]
actual=[x.item() for x in actual]

In [98]:
from sklearn.metrics import roc_auc_score,precision_score, recall_score, f1_score
from scipy.special import expit

In [99]:
roc_auc_score(trainY,preds),roc_auc_score(actual,preds_valid),

(0.9891491058761531, 0.9949269740432497)

In [100]:
f1_score(trainY,np.round(expit(preds))), f1_score(actual,np.round(expit(preds_valid)))

(0.9430490763202091, 0.9643845727994095)

In [101]:
precision_score(trainY,np.round(expit(preds))), precision_score(actual,np.round(expit(preds_valid)))

(0.9420470855779447, 0.9571428571428572)

In [102]:
recall_score(trainY,np.round(expit(preds))), recall_score(actual,np.round(expit(preds_valid)))

(0.9440532008293949, 0.9717367050948308)

In [105]:
learner_wd=learner.wd*.1

In [106]:
model_sentiment.unfreeze_embedding()

AttributeError: 'Learner' object has no attribute 'unfreeze_embedding'

In [92]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [93]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:100 0.1789311107620597  0.9311538875102997
Batch:200 0.2329713966138661  0.9111538860201835
Batch:300 0.2509736227616668  0.9042949108282725
Batch:400 0.26191826361231507  0.8983173455297947
Batch:500 0.26753545679897067  0.8957692681550979
Batch:600 0.26879613279675446  0.8953846523165703
Batch:700 0.27038188688989195  0.8940934434107372
Batch:800 0.27164643675554545  0.8934856137633324
Batch:900 0.27330568608310485  0.8929060198863348
Batch:1000 0.27197415528446434  0.8935769602656365
Batch:1100 0.27094284764067694  0.8936364008621736
Batch:1200 0.26861331348617873  0.8943429856995742
Batch:1300 0.2671618191553996  0.8947337649877255
Batch:1400 0.265266539560897  0.8957280593258994
Batch:1500 0.26431826198597747  0.8959102937777838


KeyboardInterrupt: 

In [87]:
torch.save(model_sentiment.state_dict(),f'{DATAPATH}/inter/combo_ytsm_nofreeze')
torch.save(optimizer.state_dict(),f'{DATAPATH}/inter/combo9_0.ytsm_nofreeze')
torch.save (model_sentiment,f'{DATAPATH}/inter/combo9_ytsm_nofreeze')
torch.save (optimizer,f'{DATAPATH}/inter/combo9_ytsm_nofreeze')
torch.save (learner,f'{DATAPATH}/inter/combo9_ytsm_nofreeze')

(0.9867109634551495, 0.987012987012987)

In [ ]:
|